In [61]:
from ptlpinns.odes import equations, numerical
from ptlpinns.models import model, transfer
import numpy as np
import time
import torch

In [62]:
N_ITER = 800
N = 256
N_PTL = 256

# solving for [0, 4.5] gives range ~ [0, 10] with PTL-PINNs
t_span_numerical = (0, 10)
t_eval_numerical = np.linspace(t_span_numerical[0], t_span_numerical[1], N)

t_span_PTL = (0, 4.5)
t_eval_PTL = np.linspace(t_span_PTL[0], t_span_PTL[1], N_PTL)

RK45_time, Radau_time, PTL_PINN_inverting, PTL_PINN_not_inverting = [], [], [], []

In [63]:
alpha_list = [0.2]
ic_lkv = [1.59, 0.95]
p_list = [6]
epsilon_list = [0.5]

ode = equations.lv_normalized(alpha_list[0])

### RK45 and Radau times

In [64]:
for j in range(N_ITER):

    start_RK45 = time.perf_counter()
    sol = numerical.solve_ode_equation(ode, (t_eval_numerical[0], t_eval_numerical[-1]), t_eval_numerical, ic_lkv, method="RK45")
    end_RK45 = time.perf_counter()

    start_Radau = time.perf_counter()
    sol = numerical.solve_ode_equation(ode, (t_eval_numerical[0], t_eval_numerical[-1]), t_eval_numerical, ic_lkv, method="Radau")
    end_Radau = time.perf_counter()

    RK45_time.append(end_RK45 - start_RK45)
    Radau_time.append(end_Radau - start_Radau)

In [65]:
print("RK45 time:", np.mean(RK45_time))
print("Radau time:", np.mean(Radau_time))

RK45 time: 0.00801075218249025
Radau time: 0.07001477025250664


### PTL-PINNs

In [66]:
undamped_path = "/home/dda24/PTL-PINNs/ptlpinns/models/train/undamped_k12"
undamped_name = "model_undamped_k12.pth"
undamped_model, _ = model.load_model(undamped_path, undamped_name)

12 True True True 1.0 16 [256, 256, 512]


In [67]:
# Compute latent representation: H(t) and derivatives
H_dict_undamped = transfer.compute_H_dict(undamped_model, N=N_PTL, bias=True, t_span=(t_span_PTL[0], t_span_PTL[1]))
training_log = {'w_ode': 1.5, 'w_ic': 1}

In [68]:
for j in range(N_ITER):

    _, _, TL_time_inverting = transfer.compute_perturbation_solution_LKV([epsilon_list[0]], p_list, [ic_lkv], alpha_list, H_dict_undamped, t_eval_PTL, training_log, all_p=True, comp_time=True, invert=True, w_sol = [])
    _, _, TL_time_not_inverting = transfer.compute_perturbation_solution_LKV([epsilon_list[0]], p_list, [ic_lkv], alpha_list, H_dict_undamped, t_eval_PTL, training_log, all_p=True, comp_time=True, invert=False, w_sol = [])

    PTL_PINN_inverting.append(TL_time_inverting)
    PTL_PINN_not_inverting.append(TL_time_not_inverting)

In [69]:
print("PTL-PINN-INVERT time:", np.mean(PTL_PINN_inverting))
print("PTL-PINN-NO-INVERT time:", np.mean(PTL_PINN_not_inverting))

PTL-PINN-INVERT time: 0.04288609571498796
PTL-PINN-NO-INVERT time: 0.00536405977625293
